<a href="https://colab.research.google.com/github/shhrrtnvr/rank_scrapper/blob/main/uri_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import ssl
import re
import csv
import itertools
import pandas as pd

In [2]:
def filter_number(string):
  return int(re.findall('[0-9]+', string)[0])

In [3]:
def create_context():
  ctx = ssl.create_default_context()
  ctx.check_hostname = False
  ctx.verify_mode = ssl.CERT_NONE
  return ctx

In [4]:
def scrap(url):
  headers={'User-Agent': 'Mozilla/5.0'}
  req = Request(url, headers = headers)
  html = urlopen(req, context=create_context())
  return BeautifulSoup(html, 'html.parser')


In [5]:
def getIDNameAndSolved(row):
  columns = list(row.find_all('td'))
  try:
    user, solved = columns[2].a, columns[3].get_text()
    return filter_number(user['href']), user.get_text(), filter_number(solved)
  except:
    return None


In [6]:
def make_table(soup):
  table = soup.find('table')
  body = table.find('tbody')
  rows = body.find_all('tr')
  return rows


In [7]:
def filter_table(table):
    filtered_table = list()
    for row in table:
        ret = getIDNameAndSolved(row)
        if ret:
            filtered_table.append(ret)
    return filtered_table

In [8]:
rankpage = 'https://www.urionlinejudge.com.br/judge/en/users/university/{uni}?page={page}&direction=DESC'
profilepage = 'https://www.urionlinejudge.com.br/judge/en/user/{id}'

In [11]:
data = list()
page = itertools.count(1)
while True:
  link = rankpage.format(uni='ulab', page=next(page))
  try:
    soup = scrap(link)
    table = make_table(soup)
    table = filter_table(table)
    data.extend(table)
  except:
    break

In [12]:
data.sort(key=lambda x: x[-1], reverse=True)

In [13]:
id, name, count = ([a[i] for a in data] for i in range(3))

In [14]:
df = pd.DataFrame({
    'ID' : id,
    'Name': name,
    'Sovle Count':count
})

In [15]:
pd.set_option('display.max_rows', None)
df


,ID,Name,Sovle Count
0,514900,Mohd Ataul Hakim,119
1,464318,Apon Ghosh,111
2,504748,Sadia Ahmed Bushra,100
3,469637,Rakib Ahmed,83
4,108156,yyyyy,67
5,514917,Nur Hasan Masum,62
6,529212,Bambi_18,62
7,504755,Razaan Reza,60
8,514945,Jenifar Gomes,59
9,287478,plabon,56
